In [4]:
import torch
import os
from torchvision import models
from util import get_datasets, model_train, model_test, TransitionLayer, CombinedModel
device = torch.device("cuda")
from hyperparameters import(
        PATH,
        save_path,
        learning_rate,
        batch_size,
        num_epochs,
        image_resize,
        num_each_class,
        test_ratio)

In [5]:
# Create DataLoaders if needed
train_loader, test_loader = get_datasets(PATH, num_each_class, test_ratio, batch_size, image_resize)

Time for loading data: 102.68s


In [6]:
# ResNet50+swin
name = 'Resnet50+swin_t.path'
name = os.path.join(save_path, name)
model_ResNet = models.resnet50(weights='DEFAULT')
# Remove the avgpool and fully connected layer
model_ResNet = torch.nn.Sequential(*list(model_ResNet.children())[:-2])

model_swin = models.swin_t(weights="DEFAULT")

transition_layer = TransitionLayer()

model = CombinedModel(model_ResNet, model_swin, transition_layer)
num_features = model.model_b.head.in_features
model.head = torch.nn.Linear(num_features, 10)

# load model
# model.load_state_dict(torch.load(name))

# Define loss function
criterion = torch.nn.CrossEntropyLoss()
criterion = criterion.to(device)
# Define optimizer
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model_train(model, optimizer, criterion, device, train_loader, num_epochs)
model_test(model, criterion, device, test_loader)

# Save the state dictionary of the model
torch.save(model.state_dict(), name)

Total parameters: 51810229
Trainable parameters: 51810229


100%|██████████| 438/438 [01:37<00:00,  4.49it/s]


Epoch 1/4, Loss: 1.1545, ACC: 61.2586


100%|██████████| 438/438 [01:46<00:00,  4.09it/s]


Epoch 2/4, Loss: 0.2060, ACC: 94.0639


...

Epoch 3/4, Loss: 0.1052, ACC: 97.1889


100%|██████████| 438/438 [01:37<00:00,  4.49it/s]


Epoch 4/4, Loss: 0.0922, ACC: 97.6313
Training time: 403.49s


100%|██████████| 188/188 [00:24<00:00,  7.75it/s]


Test Loss: 0.1059, Test Acc: 97.61
